In [144]:
"""
Takes raw exports from Realsense export tool and Cepton pcap conversion tool
Renames files and matches them based on timestamp. Does not modify source directory
Inputs:
    <date>/
        <camerafile>.bag
        <lidarfile>.pcap
        <lidarfile>.json
        <gpsfile>.ubx
            
Outputs:
    Export/
        image_00/
            Data/
                0000000001.png
                ...
            timestamps.txt
        pointcloud/
            Data/
                0000000001.bin
                ...
            timestamps.txt

Calibration and GPS data are currently handled separately

Command to start PTP timestamping on interface enp118s0 (Eth0)
sudo ptp4l -S -i enp118s0 -m
"""

import shutil
import pandas as pd
import numpy as np
import os
import open3d as o3d
from enum import Enum
from os import listdir
from os.path import isfile, join
from pyubx2 import ERR_IGNORE, GET, POLL, SET, UBXReader, UBX_PROTOCOL, NMEA_PROTOCOL, RTCM3_PROTOCOL
from ouster.sdk import open_source
from ouster.sdk import client
from tkinter import *
from tkinter.filedialog import askdirectory

class FileTypes(Enum):
    PCD = 0
    BIN = 1
    
OUTPUT_TYPE = FileTypes.PCD

# TODO:
# Add cell for conversion of camera .bag to frames

# Promt user for raw data directory
rawdatapath = askdirectory(title='Select raw data directory')
dirfiles = listdir(rawdatapath)
path = f"{rawdatapath}/"

# Parse date and paths
parts = rawdatapath.split('/')
date = parts[-1]
cam_frame_path = path + "Camera/"

# Required files
lidar_pcap = ''
lidar_json = ''
gps_file = ''
cam_bag = ''
imu_file = ''

# Find required files
for file in dirfiles:
    if(file.endswith('.pcap')):
       lidar_pcap = path + file
    elif(file.endswith('.json')):
        lidar_json = path + file
    elif(file.endswith('.ubx') or file.endswith('.log')):
        gps_file = path + file
    elif(file.endswith('.bag')):
        cam_bag = path + file

# Error checking for required data
if(lidar_pcap == ''):
    raise Exception('ERROR: Lidar pcap not found!')
if(lidar_json == ''):
    raise Exception('ERROR: Lidar json not found!')
if(gps_file == ''):
    #raise Exception('ERROR: GPS file not found!')
    pass
if(os.path.exists(cam_frame_path) == False and cam_bag == ''):
    raise Exception('No camera data found!')

# Destination paths
dest_path = path + "Export/"
cam_dest = dest_path + 'image_00/'
lidar_dest = dest_path + 'pointcloud/'
gps_dest = dest_path + 'navigation/'

imu_file = ''

#Use realsense sdk to convert .bag to frames if none exist yet
if(os.path.exists(cam_frame_path) == False):
    print('Camera frames not found, converting from bag...')
    os.mkdir(cam_frame_path)
    os.system(f'rs-convert -i {cam_bag} -p {cam_frame_path}')
    print('Exporting IMU data...')
    os.system(f'rs-convert -i {cam_bag} -v {cam_frame_path}')

# Reference to all files in the camera folder
cam_files = [f for f in listdir(cam_frame_path) if isfile(join(cam_frame_path, f))]
cam_files.sort()

# Find IMU data
for file_name in cam_files:
    if('imu_pose' in file_name):
        imu_file = file_name
        break

if(imu_file == ''):
    raise Exception("Error: IMU Data not found!")

print("All files found successfully")

All files found successfully


In [145]:
"""
Create dataframe of raw camera frame metadata from frame filename. [Epoch, Datetime, Filepath]
"""

# Camera metadata dataframe
cam_dataframe = pd.DataFrame()
temp_data = []

# Iterate through each file in the camera frame files
for file_name in cam_files:

    # If the file is not metadata
    if ('metadata' not in file_name) and ('imu_pose' not in file_name):
        # Get epoch from filename
        trimmedname = file_name.split('_')
        parts = trimmedname[2].split('.')

        # Append data to frame
        temp_dict = { 'Epoch' : float(parts[0] + '.' + parts[1]), 
                     'Datetime' : pd.to_datetime(float(parts[0] + '.' + parts[1]),unit='ms'),
                     'camera_path' : cam_frame_path + file_name }
        
        # Append camera frame info to running list
        temp_data.append(temp_dict)

# Keep list of new camera frames
#cam_data['Datetime'] = pd.to_datetime(cam_data['Epoch'])
cam_dataframe = pd.DataFrame(temp_data)
print(cam_dataframe.head())

          Epoch                      Datetime  \
0  1.725905e+12 2024-09-09 18:07:36.224684814   
1  1.725905e+12 2024-09-09 18:07:36.258033936   
2  1.725905e+12 2024-09-09 18:07:36.291382080   
3  1.725905e+12 2024-09-09 18:07:36.324731201   
4  1.725905e+12 2024-09-09 18:07:36.358079346   

                                         camera_path  
0  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
1  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
2  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
3  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
4  /home/khanj/Dev/PerceptionTools/DriveData/2024...  


In [146]:
"""
Parse metadata from ouster pcap into pandas dataframe
1. index frames from pcap file
2. record frame time start and end
3. destagger point representation
4. convert destaggered frame points and reflectivity to binary as float32 types
"""

# Empty lists of desired data to be combined later
frame_ids = []
timestarts = []
timeends = []
xyzr = []
filepaths = []

imu_accel = []
imu_gyro  = []

# Load .pcap and .json files
source = open_source(lidar_pcap, meta=[lidar_json], index = True)
lidar_metadata = source.metadata
print('frame count: ' + str(len(source)))

# Create binary data directory if doesnt exist
if(os.path.exists(f'{path}pointcloud/') == False):
    os.mkdir(f'{path}pointcloud/')
if(os.path.exists(f'{path}pointcloud/Data/') == False):
    os.mkdir(f'{path}pointcloud/Data/')
    print('Saved point cloud not found. Regenerating...')

# Get function from factory to project scan data to cartesian coordinates
xyzl = client.XYZLut(lidar_metadata)
packet_format = client.PacketFormat(lidar_metadata)

# Iterate through all lidar scan frames
for scan in source:
    # If the scan is valid
    if scan:
        # Grab frame id
        frame_ids.append(scan.frame_id)
        # Grab time start and end for frame
        timestarts.append(scan.timestamp[0])
        timeends.append(scan.timestamp[-1]) 
        
        # Project points to cartesian and destagger representation
        pos = client.destagger(lidar_metadata,xyzl(scan))

        # Destagger reflectivity
        ref = client.destagger(lidar_metadata,scan.field(client.ChanField.REFLECTIVITY))

        # Reshape reflectivity to match XZY's shape
        reflectivity_reshaped = np.expand_dims(ref, axis=2)

        # Concatenate XYZ and reflectivity and cast to 32-bit floats
        xyzref_array = np.concatenate((pos, reflectivity_reshaped), axis=2).astype(np.float32)
        xyzr.append(xyzref_array)

        if(OUTPUT_TYPE == FileTypes.BIN):
            extension = "bin"
            
        elif(OUTPUT_TYPE == FileTypes.PCD):
            extension = "pcd"

        # Grab filepath of point representation
        filepath = f"{path}pointcloud/Data/{scan.frame_id:010d}." + extension    
        filepaths.append(filepath)  

        # Write frame data to disk if doesn't exist
        if(os.path.exists(filepath) == False):

            # Output files as binary
            if(OUTPUT_TYPE == FileTypes.BIN):
                # Get byte representation of XYZR
                scanbytes = bytearray(xyzref_array)

                # Write byte representation to pointcloud/Data/ directory
                file = open(filepath, 'wb')
                file.write(scanbytes)    

            # Output files as PCD
            elif(OUTPUT_TYPE == FileTypes.PCD):
                # Reshape array
                xyzref_array = xyzref_array.reshape((-1,4))[:,0:3]

                # Convert to Open3D point cloud
                o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyzref_array))

                print(f'writing to: {filepath}')
                # Save to whatever format you like
                o3d.io.write_point_cloud(filepath, o3d_pcd)

            else:
                print(f'{scan.frame_id} already exists, skipping...')

# Build pandas dataframe of desired lidar scan data
lidar_dataframe = pd.DataFrame()
lidar_dataframe['lidar_id'] = pd.DataFrame(frame_ids)
lidar_dataframe['Timestart'] = pd.DataFrame(timestarts)
lidar_dataframe['Timeend'] = pd.DataFrame(timeends)
lidar_dataframe['lidar_path'] = pd.DataFrame(filepaths)

# Convert time strings to datetime objects
lidar_dataframe['Timestart'] = pd.to_datetime(lidar_dataframe['Timestart'],unit=NS)
lidar_dataframe['Timeend'] = pd.to_datetime(lidar_dataframe['Timeend'],unit=NS)

# Assign scan midpoint as frame epoch   
lidar_dataframe['Datetime'] = pd.DataFrame(lidar_dataframe['Timestart'] + ((lidar_dataframe['Timeend']-lidar_dataframe['Timestart'])/2))
print(lidar_dataframe.head())

loading metadata from ['/home/khanj/Dev/PerceptionTools/DriveData/2024-09-09/2024099_1105_OS-2-128_122414001752.json']
[2024-09-09 12:35:14.598] [ouster::sensor] [info] parsing non-legacy metadata format
 [####################] 100.0% indexed
finished building index
frame count: 2097
   lidar_id                     Timestart                       Timeend  \
0       879 1970-01-01 00:00:00.000000000 2024-09-09 18:05:01.052299345   
1       880 2024-09-09 18:05:01.052398057 2024-09-09 18:05:01.152359244   
2       881 2024-09-09 18:05:01.152453023 2024-09-09 18:05:01.252428948   
3       882 2024-09-09 18:05:01.252529521 2024-09-09 18:05:01.352468787   
4       883 2024-09-09 18:05:01.352561353 2024-09-09 18:05:01.452436919   

                                          lidar_path  \
0  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
1  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
2  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
3  /home/khanj/Dev/PerceptionTools/Drive

In [147]:
"""
Parse gps data marge with perception frame data
Written for .UBX file generated by u-center for u-blox7 USB GPS
"""
# Empty dataframe for gps data
gpsframe = pd.DataFrame()

# Data lists
times = []
latitudes = []
longitudes = []
altitudes = []
altunits = []
alttimes = []

# Parse GPS messages from .ubx file
for mode in (GET, SET, POLL):
    msgcount = 0
    ubr = None
    
    # Open file as U-Center .ubx file
    if(gps_file.endswith('.ubx')):
        print("Reading gps as U-Center output")
        with open(gps_file, "rb") as stream:
            # Create reader
            ubr = UBXReader(stream, quitonerror=ERR_IGNORE, parsing=True,msgmode=mode)

        # Iterate through messages parsed by reader
        for _, parsed in ubr:
            # If message is valid
            if parsed is not None:
                msgcount += 1

                # Grab latitude & longitude if present
                if(hasattr(parsed, 'lat')):
                    times.append(pd.to_datetime(date + str(' ' + parsed.time.strftime('%H:%M:%S'))))
                    latitudes.append(parsed.lat)
                    longitudes.append(parsed.lon)
                # Grab altitude if present
                if(hasattr(parsed,'alt')):
                    altitudes.append(parsed.alt)
                    altunits.append(parsed.altUnit)
                    alttimes.append(pd.to_datetime(date + str(' ' + parsed.time.strftime('%H:%M:%S'))))
            
    # Open file as pygpsclient .log file
    elif(gps_file.endswith('.log')):
        print("Reading gps as pygpsclient binary output")
        with open(gps_file, 'rb') as stream:
            # Create reader
            ubr = UBXReader(stream, quitonerror=ERR_IGNORE, parsing=True, protfilter=UBX_PROTOCOL|NMEA_PROTOCOL|RTCM3_PROTOCOL)

            # Iterate through messages parsed by reader
            for _, parsed in ubr:
                # If message is valid
                if parsed is not None:
                    msgcount += 1

                    # Grab latitude & longitude if present
                    if(hasattr(parsed, 'lat')):
                        times.append(pd.to_datetime(date + str(' ' + parsed.time.strftime('%H:%M:%S'))))
                        latitudes.append(parsed.lat)
                        longitudes.append(parsed.lon)
                    # Grab altitude if present
                    if(hasattr(parsed,'alt')):
                        altitudes.append(parsed.alt)
                        altunits.append(parsed.altUnit)
                        alttimes.append(pd.to_datetime(date + str(' ' + parsed.time.strftime('%H:%M:%S'))))

    # Print number of messages parsed
    print(f'\n{msgcount} {("GET","SET","POLL")[mode]} messages parsed\n\n')

# Build gps dataframe
gpsframe['Datetime'] = times
gpsframe['Latitude'] = latitudes
gpsframe['Longitude'] = longitudes

# Build separate altitude dataframe since sampling rates are different
altframe = pd.DataFrame()
altframe['Altitude'] = altitudes
altframe['Alt Unit'] = altunits
altframe['Datetime'] = alttimes

# Sort data by time
gpsframe = gpsframe.sort_values(by='Datetime')
altframe = altframe.sort_values(by='Datetime')

# Merge GPS and nearest Altitude into GPS dataframe. 0 order hold altitude
gpsframe = pd.merge_asof(gpsframe,altframe,on='Datetime',suffixes=('_gps','_alt'))
print(gpsframe.head())


0 GET messages parsed



0 SET messages parsed



0 POLL messages parsed


Empty DataFrame
Columns: [Datetime, Latitude, Longitude, Altitude, Alt Unit]
Index: []


In [148]:
"""
Add IMU data to dataframe
"""

#TODO:
#1. Change backend timestamp to global timestamp for accurate time delay correction

# open IMU file
rawimu = open(cam_frame_path + imu_file,'r')

# Empty lists for dataframe
gyro_data = []
accel_data = []

# Remove leading empty lines written by realsense tool
line = rawimu.readline()
line = rawimu.readline()    
line = rawimu.readline()

# Iterate through gyroscope data
while(line != '\n'):
    gyro_data.append(line.replace('\n','').split(','))
    line = rawimu.readline()

# Remove leading empty lines
line = rawimu.readline()
line = rawimu.readline()

# Iterate through accelerometer data
while(line):
    accel_data.append(line.replace('\n','').split(','))
    line = rawimu.readline()

# Build pandas dataframe of IMU data
gyroframe = pd.DataFrame(gyro_data[1:],columns=gyro_data[0])
gyroframe['Backend Timestamp(ms)'] = gyroframe['Backend Timestamp(ms)'].astype(np.int64).apply(lambda x: pd.to_datetime(x,unit='ms'))
imuframe = pd.DataFrame(accel_data[1:],columns=accel_data[0])
imuframe['Backend Timestamp(ms)'] = imuframe['Backend Timestamp(ms)'].astype(np.int64).apply(lambda x: pd.to_datetime(x,unit='ms'))

# Merge by backend timestamp
imuframe = pd.merge_asof(imuframe,gyroframe,on='Backend Timestamp(ms)',suffixes=('_Accel','_Gyro'))
imuframe = imuframe.rename(columns={'Backend Timestamp(ms)':'Datetime'})
print(imuframe.head())

  Stream Type_Accel F#_Accel HW Timestamp (ms)_Accel                Datetime  \
0             Accel     1442                46993958 2024-09-09 18:07:36.254   
1             Accel     1443                47003873 2024-09-09 18:07:36.264   
2             Accel     1444                47013789 2024-09-09 18:07:36.274   
3             Accel     1445                47023704 2024-09-09 18:07:36.284   
4             Accel     1446                47033620 2024-09-09 18:07:36.294   

  Host Timestamp(ms)_Accel 3DOF_x_Accel 3DOF_y_Accel 3DOF_z_Accel  \
0            1725905256254        0.137        9.248        2.942   
1            1725905256264        0.127        9.218        2.932   
2            1725905256274        0.127        9.228        2.932   
3            1725905256284        0.127        9.248        2.932   
4            1725905256294        0.137        9.257        2.932   

  Stream Type_Gyro F#_Gyro HW Timestamp (ms)_Gyro Host Timestamp(ms)_Gyro  \
0             Gyro    2840 

In [149]:
"""
find closest camera frame for each lidar frame and merge to single dataframe
Add GPS data
Add IMU data
"""

# Print number of frames to be combined
print("Cam frames: " + str(len(cam_dataframe.index)))
print("Lidar frames: " + str(len(lidar_dataframe.index)))

# Left hand SQL style join on lidar and camera frames
frame_data = pd.DataFrame()
frame_data = pd.merge_asof(lidar_dataframe.sort_values('Datetime'),cam_dataframe.sort_values('Datetime'),on='Datetime')

# Print lone lidar frames
print("Lidar frames with no matching camera: " + str(frame_data.index.size - frame_data.dropna().index.size))
frame_data = frame_data.dropna().reset_index(drop=True)

# Calculate time between camera and lidar timestamps
#frame_data['fused_frame_deltas'] = (frame_data['Datetime'] - frame_data['Datetime_y']).dt.total_seconds()
# Match with closest gps
#frame_data = pd.merge_asof(frame_data,gpsframe,on='Datetime')
# Match with closest IMU
frame_data = pd.merge_asof(frame_data,imuframe,on='Datetime')

# Save final dataframe to csv
frame_data.to_csv(f'framedata_{date}.csv')
print(frame_data.head())

Cam frames: 1776
Lidar frames: 2097
Lidar frames with no matching camera: 1554
   lidar_id                     Timestart                       Timeend  \
0      2432 2024-09-09 18:07:36.244398204 2024-09-09 18:07:36.344308955   
1      2433 2024-09-09 18:07:36.344406742 2024-09-09 18:07:36.444255669   
2      2434 2024-09-09 18:07:36.444359567 2024-09-09 18:07:36.544237671   
3      2435 2024-09-09 18:07:36.544337552 2024-09-09 18:07:36.644215850   
4      2436 2024-09-09 18:07:36.644308163 2024-09-09 18:07:36.744183120   

                                          lidar_path  \
0  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
1  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
2  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
3  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
4  /home/khanj/Dev/PerceptionTools/DriveData/2024...   

                       Datetime         Epoch  \
0 2024-09-09 18:07:36.294353579  1.725905e+12   
1 2024-09-09 18:07:36.394331205  1.

In [153]:
"""
Move camera and lidar to final dataset directory and rename based on index
"""

# Create export directory if does not exist
if (not os.path.exists(dest_path)):
    os.makedirs(dest_path)
if (not os.path.exists(lidar_dest)):
    os.makedirs(lidar_dest)
    os.makedirs(lidar_dest + 'data/')
if (not os.path.exists(cam_dest)):
    os.makedirs(cam_dest)
    os.makedirs(cam_dest + 'data/')
if (not os.path.exists(gps_dest)):
    os.makedirs(gps_dest)
if (not os.path.exists(gps_dest + 'data/')):
    os.makedirs(gps_dest + 'data/')

# Write frame timestamps to pointcloud/Data/
timepath = f'{path}pointcloud/timestamps_start.txt'
if(os.path.exists(timepath) == False):
    with open(timepath, 'a') as f:
        f.write(lidar_dataframe['Timestart'].to_string(header=False, index=False))

timepath = f'{path}pointcloud/timestamps_end.txt'
if(os.path.exists(timepath) == False):
    with open(timepath, 'a') as f:
        f.write(lidar_dataframe['Timeend'].to_string(header=False, index=False))

timepath = f'{path}pointcloud/timestamps.txt'
if(os.path.exists(timepath) == False):
    with open(timepath,'a') as f:
        f.write(lidar_dataframe['Datetime'].to_string(header=False, index=False))
lidar_dataframe.head()

# Move matching frames to dataset directory based on index in matched dataframe
for index, row in frame_data.iterrows():    
    shutil.copy(row['lidar_path'], lidar_dest + 'data/' + '{index:010d}'.format(index = index) + '.pcd')
    shutil.copy(row['camera_path'], cam_dest + 'data/' + '{index:010d}'.format(index = index) + '.png')
    #gpspath = gps_dest + 'data/' + f'{index:010d}'.format(index = index) + '.txt'
    #f = open(gpspath,'x')
    #f.write(str(row['Latitude']) + ' ' + str(row['Longitude']))
    #f.close()

# Copy lidar timestamps to export
shutil.copy(f'{path}pointcloud/timestamps_end.txt', lidar_dest)
shutil.copy(f'{path}pointcloud/timestamps_start.txt', lidar_dest)
shutil.copy(f'{path}pointcloud/timestamps.txt', lidar_dest)

'/home/khanj/Dev/PerceptionTools/DriveData/2024-09-09/Export/pointcloud/timestamps.txt'

In [ ]:
'''
Move IMU and GPS data to dataset directory
'''

